In [ ]:
! pip install yfinance

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# download ethereum data from yfinance
eth = yf.download('ETH-USD', period='max', interval='1d')

# convert to pandas dataframe
eth = pd.DataFrame(eth)

eth.head()

In [ ]:
# add a column for day of the week
eth['day_of_week'] = eth.index.dayofweek

eth

In [ ]:
# change all the values to percentage change
eth['Open'] = eth['Open'].pct_change()
eth['High'] = eth['High'].pct_change()
eth['Low'] = eth['Low'].pct_change()
eth['Close'] = eth['Close'].pct_change()
eth['Volume'] = eth['Volume'].pct_change()
eth['Adj Close'] = eth['Adj Close'].pct_change()

eth.head()

In [ ]:
eth['Adj Close'] = eth['Adj Close'].shift(-1)

eth

In [ ]:
# drop adj close column
eth = eth.drop(['Close'], axis=1)

# drop nan values
eth = eth.dropna()

In [ ]:
# calculate the number on infs
eth.isin([np.inf, -np.inf]).sum()

In [ ]:
# change all the infs to 0
eth = eth.replace([np.inf, -np.inf], 0)

In [ ]:
# split the data into x and y train and test using built-in function
from sklearn.model_selection import train_test_split

x = eth.drop(['Adj Close'], axis=1)
y = eth['Adj Close']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, shuffle=False)

# scale the data
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:
# LSTM model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

def create_model(optimizer='adam', dropout_rate=0.2, num_neurons=50):
    model = Sequential()
    model.add(LSTM(num_neurons, return_sequences=True))
    model.add(Dropout(dropout_rate))
    model.add(LSTM(num_neurons, return_sequences=False))
    model.add(Dropout(dropout_rate))
    model.add(Dense(25))
    model.add(Dense(1))
    model.compile(optimizer=optimizer, loss='mean_absolute_error')
    return model

# Create the model
model = KerasRegressor(build_fn=create_model, verbose=0)

# Define the grid search parameters
optimizer = ['Adam', 'RMSprop']
dropout_rate = [0.0, 0.1, 0.2, 0.3]
num_neurons = [50, 100, 150]
batch_size = [8, 16, 32, 64]
epochs = [10, 20, 30]
param_grid = dict(optimizer=optimizer, dropout_rate=dropout_rate, num_neurons=num_neurons, batch_size=batch_size, epochs=epochs)

# Reshape the data
x_train = x_train.reshape(x_train.shape[0], 1, x_train.shape[1])
x_test = x_test.reshape(x_test.shape[0], 1, x_test.shape[1])

# Perform the grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(x_train, y_train, validation_data=(x_test, y_test))

# Print the best parameters and the corresponding score
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

In [ ]:
# make predictions on the test set
y_pred = grid_result.predict(x_test)

# calculate the MAE
mae = mean_absolute_error(y_test, y_pred)
print("MAE:", mae)


In [ ]:
# plot the prediction and the test set with test set index
plt.figure(figsize=(20, 10))
plt.plot(y_test.index, y_test, label='Test Set')
plt.plot(y_test.index, y_pred, label='Prediction')
plt.legend()
plt.show()